In [1]:
import os 

import pandas as pd 
import numpy as np 
from tqdm import tqdm
import glob

import torch
import torch.nn as nn 
from torch.utils.data import DataLoader

from torchvision import transforms
import torch.nn.functional as F
import nibabel as nib

from sklearn.model_selection import KFold

import matplotlib.pyplot as plt 

from monai.transforms import *
from monai.utils import set_determinism, first

from monai.data import (
    DataLoader,
    Dataset,
    CacheDataset,
    load_decathlon_datalist,
    decollate_batch,
)

2024-06-06 12:58:28.217902: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-06 12:58:28.251465: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-06 12:58:28.251496: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-06 12:58:28.251514: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-06 12:58:28.257645: I tensorflow/core/platform/cpu_feature_g

In [2]:
from dp_model.model_files.multi_sfcn import MultiSFCN
from dp_model import dp_loss as dpl
from dp_model import dp_utils as dpu
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

In [3]:
csv = './IXI_train.csv'
df = pd.read_csv(csv)
df

,idx,f,age,sex,height,weight
0,1,IXI002-Guys-0828-T1.nii.gz,35.800137,2,164,58
1,2,IXI012-HH-1211-T1.nii.gz,38.781656,1,175,70
2,3,IXI013-HH-1212-T1.nii.gz,46.710472,1,182,70
3,4,IXI014-HH-1236-T1.nii.gz,34.236824,2,163,65
4,5,IXI015-HH-1258-T1.nii.gz,24.284736,1,181,90
...,...,...,...,...,...,...
495,496,IXI648-Guys-1107-T1.nii.gz,47.723477,1,193,120
496,497,IXI651-Guys-1118-T1.nii.gz,50.395619,1,175,61
497,498,IXI652-Guys-1116-T1.nii.gz,42.989733,1,163,80
498,499,IXI653-Guys-1122-T1.nii.gz,46.220397,1,172,100


In [4]:
TRAIN_CSV_PATH = './IXI_train.csv'
DATA_PATH = '/mnt/babymri/data_age_mni1mm'

In [5]:
pwd

'/home/oj/ycm'

In [6]:
# 프로젝트 폴더 생성
project_name = "multi_sfcn"
data_root = '/home/oj/ycm'
    
# for name in subdir_li:
#     subdir_path = f'{dir_path}/{name}'
#     if os.path.exists(subdir_path) == False :
#         os.mkdir(subdir_path)

## 3D_T1_reg_PD.nii.gz 있는 폴더만 가져오기

In [7]:
## 3D_T1_reg_PD.nii.gz가 nas에 있으면 가져오기

import glob

# data_pattern = '/mnt/babymri/data_age_mni1mm/*/TOF_bet.nii.gz'
data_pattern = '/mnt/babymri/data_age_mni1mm/*/T1_aseg.nii.gz'

# 해당 패턴에 매칭되는 모든 파일 경로 가져오기
all_files = glob.glob(data_pattern, recursive=True)

# '3D_T1_reg_PD.nii.gz' 파일이 있는 디렉토리만 추출
directories_with_file = [os.path.dirname(file) for file in all_files if os.path.isfile(file)]

# 결과 출력
# for directory in directories_with_file:
#     print(directory)

print(len(directories_with_file))

477


In [8]:
## csv f축 가져오기

image_paths = []
labels = []
for idx, row in df.iterrows(): # df 각 row 돌면서 index랑 row 정보(tuple) 저장
    filename = os.path.join(DATA_PATH, row['f'].replace('-T1.nii.gz', ''))

    if os.path.exists(filename):
        image_paths.append(filename)
print(len(image_paths))

488


In [9]:
## csv와 nas에 둘다 존재하는 파일 + 3D_T1_reg_PD.nii.gz 존재

exist_image = []
for image in directories_with_file:
    if image in image_paths:
        exist_image.append(image)
        
print(len(exist_image))

442


In [10]:
## image, label 가져오기

image_paths = []
tof_paths = []
mask_paths = []
labels = []
for idx, row in df.iterrows(): # df 각 row 돌면서 index랑 row 정보(tuple) 저장
    filename = os.path.join(DATA_PATH, row['f'].replace('-T1.nii.gz', ''))
#     filename = os.path.join(DATA_PATH, row['f'].replace('-T1.nii.gz', ''), 'nifti', '3D_T1_reg_PD.nii.gz')
    if filename in exist_image:
        t1name = os.path.join(filename, 'T1_bet.nii.gz')
        tofname = os.path.join(filename, 'TOF_bet.nii.gz')
        maskname = os.path.join(filename, 'T1_aseg.nii.gz')
        image_paths.append(t1name)
        tof_paths.append(tofname)
        mask_paths.append(maskname)
        labels.append(int(row['age']))
print(len(image_paths),len(tof_paths),len(mask_paths), len(labels))

442 442 442 442


In [11]:
data_ind = np.arange(0,len(image_paths))
data_dict = [
    {
        "t1": image_paths[ind],
        "tof": tof_paths[ind],
        "mask": mask_paths[ind],
        "label": labels[ind]
    }
    for ind in data_ind
]

train_Data = data_dict[88:]
val_Data = data_dict[:88]
n_train = len(train_Data)
n_valid = len(val_Data)

In [12]:
train_transform =Compose([
    LoadImaged(keys=["t1","tof"]),
    EnsureChannelFirstd(keys=["t1","tof"]),
    # RandGaussianSmoothd(
    #     keys=["t1","tof"],
    #     sigma_x=(0.25, 1.5),
    #     sigma_y=(0.25, 1.5),
    #     sigma_z=(0.25, 1.5),
    #     approx="erf",
    #     prob=0.3,
    #     allow_missing_keys=False,
    # ),
    # RandGaussianNoised(keys=["t1","tof"],
    #                        prob=0.5, mean=0, std=25, 
    #                        allow_missing_keys=False), 
    CenterSpatialCropd(
        keys=["t1","tof"],
        roi_size=(164,192,164)  
    ),
    RandSpatialCropd(keys=["t1","tof"],
                         roi_size=(160,192,160), 
                         max_roi_size=None, 
                         random_center=True, 
                         random_size=False
                        ),
    # RandFlipd(
    #         keys=["t1","tof"],
    #         spatial_axis=[0],
    #         prob=0.5,
    #     ),
    ToTensord(
        keys=["t1","tof"]
    ),
])

valid_transform = Compose([
    LoadImaged(keys=["t1","tof", "mask"]),
    EnsureChannelFirstd(keys=["t1","tof","mask"]),
    CenterSpatialCropd(
        keys=["t1","tof","mask"],
        roi_size=(160,192,160)  
    ),
    ToTensord(
        keys=["t1","tof","mask"]
    ),
])

In [13]:
train_ds = Dataset(
    data=train_Data,
    transform=valid_transform,
)
train_loader = DataLoader(
    train_ds, batch_size=2, shuffle=True,
)

In [14]:
val_ds = Dataset(
    data=val_Data,
    transform=valid_transform,
)
valid_loader = DataLoader(
    val_ds, batch_size=1, shuffle=False, 
)

In [15]:
all_ds = Dataset(
    data=data_dict,
    transform=valid_transform,
)
all_loader = DataLoader(
    all_ds, batch_size=1, shuffle=False, 
)

In [49]:
up_t1_tof_strange_idx = [55] # combine 결과 값이 실제 label 이상인 경우
up_t1_strange_idx = [149, 202] # t1 model 결과 값이 실제 label 이상인 경우
up_tof_strange_idx = [0, 6, 7, 14, 25, 44, 52, 55, 64, 67, 147] # tof mdoel 결과 실제 label 이상인 경우

down_t1_tof_strange_idx = [10, 46, 300, 316, 410]
down_t1_strange_idx = [46, 148, 253, 269, 295, 296, 314, 316, 322, 366]
down_tof_strange_idx = [10, 46, 59, 121, 142, 316, 410]

In [50]:
def filter_data_by_indices(data_dict, indices):
    filtered_data = [data_dict[i] for i in indices]
    return filtered_data

In [51]:
up_t1_tof_strange_data = filter_data_by_indices(data_dict, up_t1_tof_strange_idx)
up_t1_strange_data = filter_data_by_indices(data_dict, up_t1_strange_idx)
up_tof_strange_data = filter_data_by_indices(data_dict, up_tof_strange_idx)

down_t1_tof_strange_data = filter_data_by_indices(data_dict, down_t1_tof_strange_idx)
down_t1_strange_data = filter_data_by_indices(data_dict, down_t1_strange_idx)
down_tof_strange_data = filter_data_by_indices(data_dict, down_tof_strange_idx)

print(len(up_t1_tof_strange_data), len(up_t1_strange_data), len(up_tof_strange_data))
print(len(down_t1_tof_strange_data), len(down_t1_strange_data), len(down_tof_strange_data))

1 2 11
5 10 7


In [52]:
up_t1_tof_ds = Dataset(
    data=up_t1_tof_strange_data,
    transform=valid_transform,
)
up_t1_tof_loader = DataLoader(
    up_t1_tof_ds, batch_size=1, shuffle=False, 
)
up_t1_ds = Dataset(
    data=up_t1_strange_data,
    transform=valid_transform,
)
up_t1_loader = DataLoader(
    up_t1_ds, batch_size=1, shuffle=False, 
)
up_tof_ds = Dataset(
    data=up_tof_strange_data,
    transform=valid_transform,
)
up_tof_loader = DataLoader(
    up_tof_ds, batch_size=1, shuffle=False, 
)

In [53]:
down_t1_tof_ds = Dataset(
    data=down_t1_tof_strange_data,
    transform=valid_transform,
)
down_t1_tof_loader = DataLoader(
    down_t1_tof_ds, batch_size=1, shuffle=False, 
)
down_t1_ds = Dataset(
    data=down_t1_strange_data,
    transform=valid_transform,
)
down_t1_loader = DataLoader(
    down_t1_ds, batch_size=1, shuffle=False, 
)
down_tof_ds = Dataset(
    data=down_tof_strange_data,
    transform=valid_transform,
)
down_tof_loader = DataLoader(
    down_tof_ds, batch_size=1, shuffle=False, 
)

In [97]:
for step, batch in enumerate(down_tof_loader):
    labels = batch["label"]
    print(labels)

tensor([74])
tensor([52])
tensor([51])
tensor([83])
tensor([63])
tensor([86])
tensor([83])


In [54]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MultiSFCN(nn.Module):
    def __init__(self, channel_number=[32, 64, 128, 256, 256, 64], output_dim=40, dropout=True):
        super(MultiSFCN, self).__init__()
        n_layer = len(channel_number)
        
        # Define encoders for each contrast
        self.encoder1 = self._make_encoder(channel_number)
        self.encoder2 = self._make_encoder(channel_number)
        
        # Classifier
        self.x1_classifier = self._make_classifier(channel_number[-1], output_dim, dropout)
        self.x2_classifier = self._make_classifier(channel_number[-1], output_dim, dropout)
        self.classifier = self._make_classifier(128, output_dim, dropout)  # Multiply by 2 for concatenated features

    def _make_encoder(self, channel_number):
        encoder = nn.Sequential()
        n_layer = len(channel_number)
        for i in range(n_layer):
            if i == 0:
                in_channel = 1  # Input channels for the first layer
            else:
                in_channel = channel_number[i-1]  # Input channels for subsequent layers
            out_channel = channel_number[i]
            if i < n_layer-1:
                encoder.add_module('conv_%d' % i, self.conv_layer(in_channel,
                                                                       out_channel,
                                                                       maxpool=True,
                                                                       kernel_size=3,
                                                                       padding=1))
            else:
                encoder.add_module('conv_%d' % i, self.conv_layer(in_channel,
                                                                       out_channel,
                                                                       maxpool=False,
                                                                       kernel_size=1,
                                                                       padding=0))
        return encoder
    def _upsample(self, in_channels, out_channels):
        # Define upsampling layers
        upsample = nn.Sequential(
            nn.ConvTranspose3d(in_channels=40, out_channels=64, kernel_size=(5,6,5), stride=1, padding=0),
            nn.ReLU()
        )
        # Apply upsampling
        return upsample
        
    def _make_combine_conv(self, in_channels, out_channels):
        layer = nn.Sequential(
            nn.Conv3d(in_channels, out_channels, kernel_size=1, padding=0),
            nn.BatchNorm3d(out_channels),
            nn.ReLU()
        )

        return layer
        
    def _make_classifier(self, in_channels, output_dim, dropout):
        classifier = nn.Sequential()
        avg_shape = [5, 6, 5]
        classifier.add_module('average_pool', nn.AvgPool3d(avg_shape))
        if dropout is True:
            classifier.add_module('dropout', nn.Dropout(0.5))
        classifier.add_module('conv', nn.Conv3d(in_channels, output_dim, padding=0, kernel_size=1))
        return classifier

    @staticmethod
    def conv_layer(in_channel, out_channel, maxpool=True, kernel_size=3, padding=0, maxpool_stride=2):
        if maxpool is True:
            layer = nn.Sequential(
                nn.Conv3d(in_channel, out_channel, padding=padding, kernel_size=kernel_size),
                nn.BatchNorm3d(out_channel),
                nn.MaxPool3d(2, stride=maxpool_stride),
                nn.ReLU()
            )
        else:
            layer = nn.Sequential(
                nn.Conv3d(in_channel, out_channel, padding=padding, kernel_size=kernel_size),
                nn.BatchNorm3d(out_channel),
                nn.ReLU()
            )
        return layer

    def forward(self, x):
        # Forward pass through each encoder
        x1 = x[:, 0, ...]
        x2 = x[:, 1, ...]
        x1_f = self.encoder1(x1) #[bs, 64, 5, 6, 5]
        x2_f = self.encoder2(x2)

        combined_features = torch.cat((x1_f, x2_f), dim=1) # [bs, 128, 5, 6, 5]

        output = self.classifier(combined_features) # [bs, 40, 1, 1, 1]
        x1_out = self.x1_classifier(x1_f)
        x2_out = self.x2_classifier(x2_f)
        
        output = F.log_softmax(output, dim=1)
        x1_out = F.log_softmax(x1_out, dim=1)
        x2_out = F.log_softmax(x2_out, dim=1)
        
        return output, x1_out, x2_out


In [55]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"GPU CHECK : {True if device.type == 'cuda' else False}")

model = MultiSFCN()
model = torch.nn.DataParallel(model)
model = model.to(device)

model

GPU CHECK : True


DataParallel(
  (module): MultiSFCN(
    (encoder1): Sequential(
      (conv_0): Sequential(
        (0): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (3): ReLU()
      )
      (conv_1): Sequential(
        (0): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (3): ReLU()
      )
      (conv_2): Sequential(
        (0): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (1): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil

In [56]:
bin_range = [12,92]
bin_step = 2
sigma = 2

In [57]:
label = [20,80]
y, bc = dpu.num2vect(label, bin_range, bin_step, sigma)
y = torch.tensor(y, dtype=torch.float32)

In [40]:
model.load_state_dict(torch.load("./new_multi_sfcn/1/models/model_4.699.pth", map_location=device))

<All keys matched successfully>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

num_rows = 6
num_cols = 6
num_slices = num_rows * num_cols

threshold_value = 0.1

plt.figure(figsize=(20, 20))
for i in range(num_slices):
    slice_num = i * 5

    if slice_num > 155:
        break

    rotated_x = np.rot90(x1[0, 0, :, :, slice_num], k=1)
    rotated_mask = np.rot90(mask[0, 0, :, :, slice_num], k=1)
    rotated_cam = np.rot90(final_grayscale_cam[0, :, :, slice_num], k=1)
    rotated_cam_threshold = np.where(rotated_cam > threshold_value, 100, 0)

    overlapping_indices = (rotated_cam_threshold == 100)
    
    overlapping_mask_values = rotated_mask.copy()
    overlapping_mask_values[~overlapping_indices] = 0

    if overlapping_mask_values.size > 0:
        plt.subplot(num_rows, num_cols, i + 1)
        plt.imshow(rotated_x, cmap='gray')
        plt.imshow(overlapping_mask_values, cmap="jet",alpha=0.2)
        plt.title(f'Slice {slice_num}')
        plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
import nibabel as nib

nifti_path = '/mnt/babymri/data_age_mni1mm/IXI002-Guys-0828/T1_bet.nii.gz'
nft = nib.load(nifti_path)
data = nft.get_fdata()

affine=nft.affine
header=nft.header

th = 0.1

mask_tmp = torch.tensor(mask[0, 0, :, :, :])
cam_tmp = torch.tensor(final_grayscale_cam[0])
cam_threshold = np.where(cam_tmp > th, 1, 0)

overlapping_indices = (cam_threshold == 1)

overlapping_masks = mask_tmp
overlapping_masks[~overlapping_indices] = 0

print(overlapping_masks.shape)

mask_values = overlapping_masks.flatten()
unique_values = np.unique(mask_values.cpu())

print("mask 배열의 값들:")
print(unique_values)

nib.save(nib.Nifti1Image(overlapping_masks.cpu(), affine=affine, header=header), './GradCam/T1/60_T1_freesurf_old_segmask.nii.gz')

In [ ]:
import nibabel as nib

cam_nft = nib.load('./GradCam/T1/T1_20_final_grayscale_cam.nii.gz')
final_grayscale_cam = cam_nft.get_fdata()

print(final_grayscale_cam.shape)
x1 = nib.load('./T1_bet_cropped.nii.gz')
x1 = x1.get_fdata()
print(x1.shape)

for batch in train_loader:
    x2, mask, age = (batch["tof"]).to(device), (batch["mask"]).to(device), batch["label"]
    break

# mask = nib.load('./GradCam/T1/20_T1_freesurf_segmask.nii.gz')
# mask = mask.get_fdata()
# print(mask.shape)

In [ ]:
threshold_value = 0.095

rotated_x = np.rot90(x1[:, :, 50], k=1)
rotated_mask = np.rot90(mask[0,0,:, :, 50], k=1)
rotated_cam = np.rot90(final_grayscale_cam[:, :, 50], k=1)
rotated_cam_threshold = np.where(rotated_cam < threshold_value, 0, rotated_cam)

overlapping_indices = (rotated_mask != 0) & (rotated_cam_threshold != 0)

masked_cam = np.ma.masked_where(~overlapping_indices, rotated_cam)

plt.imshow(rotated_x, cmap='gray')
plt.imshow(masked_cam, cmap='jet', alpha=0.2)
plt.axis('off')

In [ ]:
threshold_value = 0.1

rotated_x = np.rot90(x1[:, 65, :], k=1)
rotated_mask = np.rot90(mask[0,0,:, 65, :], k=1)
rotated_cam = np.rot90(final_grayscale_cam[:, 65, :], k=1)
rotated_cam_threshold = np.where(rotated_cam < threshold_value, 0, rotated_cam)

overlapping_indices = (rotated_mask != 0) & (rotated_cam_threshold != 0)

masked_cam = np.ma.masked_where(~overlapping_indices, rotated_cam)

plt.imshow(rotated_x, cmap='gray')
plt.imshow(masked_cam, cmap='jet', alpha=0.2)
plt.axis('off')

In [ ]:
threshold_value = 0.1

rotated_x = np.rot90(x1[75, :, :], k=1)
rotated_mask = np.rot90(mask[0,0,75, :, :], k=1)
rotated_cam = np.rot90(final_grayscale_cam[75, :, :], k=1)
rotated_cam_threshold = np.where(rotated_cam < threshold_value, 0, rotated_cam)

overlapping_indices = (rotated_mask != 0) & (rotated_cam_threshold != 0)

masked_cam = np.ma.masked_where(~overlapping_indices, rotated_cam)

plt.imshow(rotated_x, cmap='gray')
plt.imshow(masked_cam, cmap='jet', alpha=0.2)
plt.axis('off')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

num_rows = 6
num_cols = 6
num_slices = num_rows * num_cols

threshold_value = 0.095

plt.figure(figsize=(20, 20))
for i in range(num_slices):
    slice_num = i * 5

    if slice_num > 155:
        break

    rotated_x = np.rot90(x1[:, :, slice_num], k=1)
    rotated_mask = np.rot90(mask[0,0,:, :, slice_num], k=1)
    rotated_cam = np.rot90(final_grayscale_cam[:, :, slice_num], k=1)
    rotated_cam_threshold = np.where(rotated_cam < threshold_value, 0, rotated_cam)

    overlapping_indices = (rotated_mask != 0) & (rotated_cam_threshold != 0)
    
    masked_cam = np.ma.masked_where(~overlapping_indices, rotated_cam)

    plt.subplot(num_rows, num_cols, i + 1)
    plt.imshow(rotated_x, cmap='gray')
    plt.imshow(masked_cam, cmap='jet', alpha=0.2)
    plt.title(f'Slice {slice_num}')
    plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

num_rows = 7
num_cols = 7
num_slices = num_rows * num_cols

threshold_value = 0.1

plt.figure(figsize=(20, 20))
for i in range(num_slices):
    slice_num = 10 + i * 5

    if slice_num > 190:
        break
    
    rotated_x = np.rot90(x1[:, slice_num, :], k=1)
    rotated_mask = np.rot90(mask[0, 0, :, slice_num, :], k=1)
    rotated_cam = np.rot90(final_grayscale_cam[:, slice_num, :], k=1)
    rotated_cam_threshold = np.where(rotated_cam < threshold_value, 0, rotated_cam)
    
    overlapping_indices = (rotated_mask != 0) & (rotated_cam_threshold != 0)
    
    masked_cam = np.ma.masked_where(~overlapping_indices, rotated_cam)
    
    plt.subplot(num_rows, num_cols, i + 1)
    plt.imshow(rotated_x, cmap='gray')
    plt.imshow(masked_cam, cmap='jet', alpha=0.2)
    plt.title(f'Slice {slice_num}')
    plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

num_rows = 6
num_cols = 6
num_slices = num_rows * num_cols

threshold_value = 0.1

plt.figure(figsize=(20, 20))
for i in range(num_slices):
    slice_num = i * 5

    if slice_num > 155:
        break
    
    rotated_x = np.rot90(x1[slice_num, :, :], k=1)
    rotated_mask = np.rot90(mask[0, 0, slice_num, :, :], k=1)
    rotated_cam = np.rot90(final_grayscale_cam[slice_num, :, :], k=1)
    rotated_cam_threshold = np.where(rotated_cam < threshold_value, 0, rotated_cam)
    
    overlapping_indices = (rotated_mask != 0) & (rotated_cam_threshold != 0)
    
    masked_cam = np.ma.masked_where(~overlapping_indices, rotated_cam)
    
    plt.subplot(num_rows, num_cols, i + 1)
    plt.imshow(rotated_x, cmap='gray')
    plt.imshow(masked_cam, cmap='jet', alpha=0.2)
    plt.title(f'Slice {slice_num}')
    plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
import nibabel as nib

nifti_path = '/mnt/babymri/data_age_mni1mm/IXI002-Guys-0828/T1_bet.nii.gz'
nft = nib.load(nifti_path)
data = nft.get_fdata()

affine=nft.affine
header=nft.header

nib.save(nib.Nifti1Image(final_grayscale_cam[0], affine=affine, header=header), './GradCam/T1/T1_old_train_final_grayscale_cam.nii.gz')


In [ ]:
print(final_grayscale_cam[0].shape)

In [ ]:
import nibabel as nib

nifti_path = '/mnt/babymri/data_age_mni1mm/IXI002-Guys-0828/T1_bet.nii.gz'
nft = nib.load(nifti_path)
data = nft.get_fdata()
print(data.shape)
affine=nft.affine
header=nft.header

# cropper = CenterSpatialCrop(roi_size=(160, 192, 160))
# resampled_data = cropper(np.expand_dims(data, axis=0))

# resampled_nifti = nib.Nifti1Image(resampled_data, affine=nft.affine, header=nft.header)
# nib.save(resampled_nifti, 'T1_bet_cropped.nii.gz')

tmp = nib.load('./GradCam/T1/T1_old_train_final_grayscale_cam.nii.gz')
tmpd = tmp.get_fdata()

threshold_value = 0.07
mask = np.where(tmpd > threshold_value, 1, 0)
print(mask.shape)

nib.save(nib.Nifti1Image(mask, affine=affine, header=header), './GradCam/T1/T1_old_train_cam_mask.nii.gz')

In [ ]:
import nibabel as nib

nifti_path = '/mnt/babymri/data_age_mni1mm/IXI002-Guys-0828/T1_bet.nii.gz'
nft = nib.load(nifti_path)
data = nft.get_fdata()
print(data.shape)
affine=nft.affine
header=nft.header

# cropper = CenterSpatialCrop(roi_size=(160, 192, 160))
# resampled_data = cropper(np.expand_dims(data, axis=0))

# resampled_nifti = nib.Nifti1Image(resampled_data, affine=nft.affine, header=nft.header)
# nib.save(resampled_nifti, 'T1_bet_cropped.nii.gz')

tmp = nib.load('./GradCam/T1/T1_young_train_final_grayscale_cam.nii.gz')
tmpd = tmp.get_fdata()

threshold_value = 0.07
mask = np.where(tmpd > threshold_value, 1, 0)

nib.save(nib.Nifti1Image(mask, affine=affine, header=header), './GradCam/T1/T1_young_train_cam_mask.nii.gz')

In [ ]:
# import numpy as np
# import nibabel as nib

# m1 = nib.load('./20_cam_mask.nii.gz')
# m2 = nib.load('./40_cam_mask.nii.gz')
# m3 = nib.load('./60_cam_mask.nii.gz')

# affine = m1.affine
# header = m1.header

# data1 = m1.get_fdata()
# data2 = m2.get_fdata()
# data3 = m3.get_fdata()

# total_mask_data = np.zeros_like(data1)

# total_mask_data[data1 > 0] = 1
# total_mask_data[data2 > 0] = 2
# total_mask_data[data3 > 0] = 3

# nib.save(nib.Nifti1Image(total_mask_data, affine=affine, header=header), 'total_mask.nii.gz')


In [ ]:
import numpy as np
import nibabel as nib

m1 = nib.load('./GradCam/T1/T1_young_train_cam_mask.nii.gz')
m2 = nib.load('./GradCam/T1/T1_old_train_cam_mask.nii.gz')

affine = m1.affine
header = m1.header

data1 = m1.get_fdata()
print(data1.shape)
data2 = m2.get_fdata()
print(data2.shape)

total_mask_data = np.zeros_like(data1)

total_mask_data[data1 > 0] = 1
total_mask_data[data2 > 0] = 2

nib.save(nib.Nifti1Image(total_mask_data, affine=affine, header=header), './GradCam/T1/T1_train_two_range_mask.nii.gz')


## Volume 구하기

In [123]:
down_vol_idx = [300]

In [124]:
strange_vol_list = []
strange_age_list = []
strange_img_list = []

vol_list = []
age_list = []
img_list = []

left_label_list = [17, 10, 8, 3]
right_label_list = [53, 49, 47, 42]

cnt = 0

for step, batch in enumerate(all_loader):
    x1, x2, mask, age = (batch["t1"]).to(device), (batch["tof"]).to(device), (batch["mask"]).to(device), batch["label"]
    img_name = data_dict[step]['t1']
    nft = nib.load(img_name)
    
    affine=nft.affine
    header=nft.header
    voxel_size = nft.header.get_zooms()
    if step in down_vol_idx:
        x1 = x1 / x1.mean()
        x2 = x2 / x2.mean()
        x = torch.stack((x1, x2), dim=1)

        mask_tmp = torch.tensor(mask[0, 0, :, :, :])

        left_vol = 0.
        right_vol = 0.
        tmp_vol_list = []
        
        for left_label, right_label in zip(left_label_list, right_label_list):
            left_mask = np.zeros_like(mask_tmp.cpu())
            left_mask[(mask_tmp.cpu()==left_label)] = 1
            left_vol = left_mask.sum() * voxel_size[0] * voxel_size[1] * voxel_size[2]
            right_mask = np.zeros_like(mask_tmp.cpu())
            right_mask[(mask_tmp.cpu()==right_label)] = 1
            right_vol = right_mask.sum() * voxel_size[0] * voxel_size[1] * voxel_size[2]
            total_vol = left_vol + right_vol
            tmp_vol_list.append(total_vol)
    
        strange_vol_list.append(tmp_vol_list)
        strange_age_list.append(age)
        strange_img_list.append(img_name)
    else:
        if (age >= 38 and age <= 48):
            x1 = x1 / x1.mean()
            x2 = x2 / x2.mean()
            x = torch.stack((x1, x2), dim=1)
    
            mask_tmp = torch.tensor(mask[0, 0, :, :, :])
    
            left_vol = 0.
            right_vol = 0.
            tmp_vol_list = []
        
            for left_label, right_label in zip(left_label_list, right_label_list):
                left_mask = np.zeros_like(mask_tmp.cpu())
                left_mask[(mask_tmp.cpu()==left_label)] = 1
                left_vol = left_mask.sum() * voxel_size[0] * voxel_size[1] * voxel_size[2]
                right_mask = np.zeros_like(mask_tmp.cpu())
                right_mask[(mask_tmp.cpu()==right_label)] = 1
                right_vol = right_mask.sum() * voxel_size[0] * voxel_size[1] * voxel_size[2]
                total_vol = left_vol + right_vol
                tmp_vol_list.append(total_vol)

            vol_list.append(tmp_vol_list)
            age_list.append(age)
            img_list.append(img_name)
            


/tmp/ipykernel_2888380/1737483120.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mask_tmp = torch.tensor(mask[0, 0, :, :, :])
/tmp/ipykernel_2888380/1737483120.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mask_tmp = torch.tensor(mask[0, 0, :, :, :])


In [125]:
print(strange_vol_list)
hippo = 0
thalamus = 0
cortex_small = 0
cortex_big = 0
for li in vol_list:
    hippo += li[0]
    thalamus += li[1]
    cortex_small += li[2]
    cortex_big += li[3]
length = len(vol_list)
print(length)
print(hippo/length, thalamus/length, cortex_small/length, cortex_big/length)
print(strange_vol_list[0][2] + strange_vol_list[0][3], cortex_small/length + cortex_big/length)

[[11516.0, 20709.0, 128828.0, 673100.0]]
79
11276.962025316456 18799.354430379746 134308.48101265822 639212.7974683545
801928.0 773521.2784810127


In [ ]:
age_list = [int(age.item()) for age in age_list]
print(age_list)

In [ ]:
print(left_list[2])
print(right_list[2])
print(vol_list[2])
print(age_list[2])
print(img_list[2])

In [ ]:
print(len(vol_list))

In [ ]:
combined = list(zip(age_list, img_list, vol_list))

combined.sort()

sorted_age_list, sorted_img_list, sorted_vol_list = zip(*combined)

sorted_age_list = list(sorted_age_list)
sorted_img_list = list(sorted_img_list)
sorted_vol_list = list(sorted_vol_list)

In [ ]:
new_img_list = []
for path in sorted_img_list:
    target_part = path.split('/')[-2]
    new_img_list.append(target_part)

In [ ]:
import pandas as pd

# 데이터 프레임 생성
data = {
    'age': sorted_age_list,
    'cortex_vol': sorted_vol_list
}

df = pd.DataFrame(data)

# new_img_list를 인덱스로 설정
df.set_index(pd.Index(new_img_list), inplace=True)

# 데이터프레임을 CSV 파일로 저장
df.to_csv('./Cortex_sorted_data.csv')


In [ ]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv("hippo_sorted_data.csv")

# age와 hippocampus_vol 열을 추출하여 리스트에 저장
sorted_age_list = df['age'].tolist()
sorted_vol_list = df['hippocampus_vol'].tolist()

# 결과 확인
print("Sorted Age List:", sorted_age_list)
print("Sorted Volume List:", sorted_vol_list)


In [ ]:
import numpy as np

# Scatter plot 그리기
plt.scatter(sorted_age_list, sorted_vol_list, label='Volume')

# Volume 변화량 계산
volume_changes = [sorted_vol_list[i+1] - sorted_vol_list[i] for i in range(len(sorted_vol_list)-1)]

# Age와 Volume 변화량으로 선형 근사 계산
x = np.array(sorted_age_list[:-1])
y = np.array(volume_changes)
p = np.polyfit(sorted_age_list, sorted_vol_list, 1)

# Age 범위 생성
x_range = np.linspace(min(x), max(x), 100)

# 선형 근사에 따른 y 값 생성
y_fit = np.polyval(p, x_range)

# 부드럽게 그려진 선 그래프 추가
plt.plot(x_range, y_fit, '-r', label='Trendline')

# 그래프 제목, 라벨 등 추가
plt.title('Hippo Volumes')
plt.xlabel('Age')
plt.ylabel('Volume')
plt.legend()

# 그래프 출력
plt.show()


In [ ]:
import pandas as pd

# CSV 파일 읽기 (파일 이름을 "data.csv"로 가정)
df = pd.read_csv('./Volume_sort/hippo_sorted_data.csv')

# 각 연령대별 부피의 평균값을 계산
age_20_39 = df[(df['age'] >= 20) & (df['age'] <= 39)]['hippocampus_vol'].mean()
age_40_59 = df[(df['age'] >= 40) & (df['age'] <= 59)]['hippocampus_vol'].mean()
age_60_plus = df[df['age'] >= 60]['hippocampus_vol'].mean()

# 평균값 출력
print(f"20-39세의 부피 평균값: {age_20_39}")
print(f"40-59세의 부피 평균값: {age_40_59}")
print(f"60세 이상의 부피 평균값: {age_60_plus}")


In [ ]:
import pandas as pd

# CSV 파일 읽기 (파일 이름을 "data.csv"로 가정)
df = pd.read_csv('./Volume_sort/thalamus_sorted_data.csv')

# 각 연령대별 부피의 평균값을 계산
age_20_39 = df[(df['age'] >= 20) & (df['age'] <= 39)]['thalamus_vol'].mean()
age_40_59 = df[(df['age'] >= 40) & (df['age'] <= 59)]['thalamus_vol'].mean()
age_60_plus = df[df['age'] >= 60]['thalamus_vol'].mean()

# 평균값 출력
print(f"20-39세의 부피 평균값: {age_20_39}")
print(f"40-59세의 부피 평균값: {age_40_59}")
print(f"60세 이상의 부피 평균값: {age_60_plus}")


In [ ]:
import pandas as pd

# CSV 파일 읽기 (파일 이름을 "data.csv"로 가정)
df = pd.read_csv('./Volume_sort/Cortex_sorted_data.csv')

# 각 연령대별 부피의 평균값을 계산
age_20_39 = df[(df['age'] >= 20) & (df['age'] <= 39)]['cortex_vol'].mean()
age_40_59 = df[(df['age'] >= 40) & (df['age'] <= 59)]['cortex_vol'].mean()
age_60_plus = df[df['age'] >= 60]['cortex_vol'].mean()

# 평균값 출력
print(f"20-39세의 부피 평균값: {age_20_39}")
print(f"40-59세의 부피 평균값: {age_40_59}")
print(f"60세 이상의 부피 평균값: {age_60_plus}")


In [ ]:
import pandas as pd

# CSV 파일 읽기 (파일 이름을 "data.csv"로 가정)
df = pd.read_csv('./Volume_sort/Cere_Cortex_sorted_data.csv')

# 각 연령대별 부피의 평균값을 계산
age_20_39 = df[(df['age'] >= 20) & (df['age'] <= 39)]['cortex_vol'].mean()
age_40_59 = df[(df['age'] >= 40) & (df['age'] <= 59)]['cortex_vol'].mean()
age_60_plus = df[df['age'] >= 60]['cortex_vol'].mean()

# 평균값 출력
print(f"20-39세의 부피 평균값: {age_20_39}")
print(f"40-59세의 부피 평균값: {age_40_59}")
print(f"60세 이상의 부피 평균값: {age_60_plus}")
